1. Потребуется установить Firefox и данный плагин для него для корректной работы скрипта:
https://addons.mozilla.org/ru/firefox/addon/ublock-origin/?utm_source=addons.mozilla.org&utm_medium=referral&utm_content=search

2. Затем нужно скачать актуальную версию geckodriver c https://github.com/mozilla/geckodriver/releases.
    Например сейчас это `geckodriver-v0.30.0-win64.zip`. Распаковать в любое место, будет необходимо для дальнейшего использования.

3. Выполнить в **powershell**: 
>`setx path "%path%;C:\path\to\folder\with\geckodriver\"` 

где `\path\to\folder\with\geckodriver\` заменить на путь до папки, в которой расположен `geckodriver.exe`.
После этого потребуется перезагрузка <em>(из-за добавления в PATH)</em> и программа будет готова к работе.

In [ ]:
!pip install selenium==4.0.0
!pip install tqdm==4.62.3
!pip install pandas==1.3.4

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from tqdm import tqdm
import pandas as pd
import re
import time
import os

import tickers

In [2]:
def getListOfTicketNames(dict):
    list = []
    for key in dict.keys():
        list.append(key)
          
    return list

In [3]:
def getIncomeStatement(link):
    if (link == "https://www.investing.com/equities/linde-plc"):
        resultIncomeStatementLink = link + "-income-statement?cid=942017"
    else:
        resultIncomeStatementLink = link + "-income-statement"

    return resultIncomeStatementLink

In [4]:
def getBalanceSheet(link):
    if (link == "https://www.investing.com/equities/linde-plc"):
        resultBalanceSheetLink = link + "-balance-sheet?cid=942017"
    else:
        resultBalanceSheetLink = link + "-balance-sheet"

    return resultBalanceSheetLink

In [5]:
def getCashFlow(link):
    if (link == "https://www.investing.com/equities/linde-plc"):
        resultCashFlowLink = link + "-cash-flow?cid=942017"
    else:
        resultCashFlowLink = link + "-cash-flow"
    return resultCashFlowLink

In [6]:
res_list_income = {}
res_list_balance = {}
res_list_cash = {}

links = tickers.getTicketLinks()
tkts = getListOfTicketNames(links)
specific_tkts = ['BAC', 'BA', 'COF', 'CMA', 'DFS', 'FITB', 'FRC', 'HBAN', 'JPM', 'KEY', 'MTB', 'PBCT', 'PNC',
                 'RF', 'SIVB', 'SYB', 'TFC', 'USB', 'WFC', 'ZION']

In [7]:
# Для проверки правильности работы команды. Должно быть на выходе: 'C:\\Users\\*имя пользователя*'
os.path.join(os.path.expandvars("%userprofile%"))

'%userprofile%'

In [19]:
driver = webdriver.Firefox()
extension_dir = (os.path.join(os.path.expandvars("%userprofile%")) + '\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\vg3v8nm4.default-release\\extensions\\')
extensions = [
    'uBlock0@raymondhill.net.xpi',
    ]
for extension in extensions:
    driver.install_addon(extension_dir + extension, temporary=None)
    
print("driver ready")   

tick_num = 0
for i in tqdm(links):
    
# ----------------------Income Statement---------------------------
    driver.get(getIncomeStatement(links[i]))
    time.sleep(5)

    res_list_income[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

    n = 0
    for el_inc in res_list_income[tkts[tick_num]]:
        res_list_income[tkts[tick_num]][n] = el_inc.text
        n+=1
        
# -----------------------Balance sheet------------------------------
    driver.get(getBalanceSheet(links[i]))
    time.sleep(5)    
    
    res_list_balance[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

    n = 0
    for el_bal in res_list_balance[tkts[tick_num]]:
        res_list_balance[tkts[tick_num]][n] = el_bal.text
        n+=1
            
# -------------------------Cash Flow--------------------------------
    driver.get(getCashFlow(links[i]))
    time.sleep(5)
    
    res_list_cash[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

    n = 0
    for el_cash in res_list_cash[tkts[tick_num]]:
        res_list_cash[tkts[tick_num]][n] = el_cash.text
        n+=1
        
        
    tick_num += 1
    time.sleep(3)
    
    
driver.close()
print('close')

driver ready


  0%|          | 0/505 [00:00<?, ?it/s]<ipython-input-19-7fa5fd82b5ff>:36: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res_list_income[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')
<ipython-input-19-7fa5fd82b5ff>:56: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res_list_balance[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')
<ipython-input-19-7fa5fd82b5ff>:76: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  res_list_cash[tkts[tick_num]] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')
100%|██████████| 505/505 [3:11:51<00:00, 22.80s/it]  


close


res_list_income['MMM'] -- лист строк по тикеру <br>
res_list_income['MMM'][0] -- вывыести конкреткую строку по тикеру

In [20]:
patterns = [
    re.compile("(?:^Revenue\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:^Cost of Revenue, Total\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Assets\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:^Net Income\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Accumulated Depreciation, Total\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Capital Expenditures\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Revenue\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Current Assets\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Current Liabilities\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Cash and Short Term Investments\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Notes Payable/Short Term Debt\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Payable[/]Accrued\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Other Liabilities, Total\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Net Income Before Taxes\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Changes in Working Capital\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Retained Earnings [(]Accumulated Deficit[)]\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Diluted Normalized EPS\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Equity\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Common Shares Outstanding\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Goodwill, Net\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Intangibles, Net\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)")
]

specific_patterns = [
    re.compile("(?:^Interest Income, Bank\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Interest Expense\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Non-Interest Income, Bank\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Non-Interest Expense, Bank\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Total Liabilities\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Cash & Due from Banks\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Other Earning Assets, Total\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)"),
    re.compile("(?:Net Loans\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?\s-?\d{0,}[.]?([0-9]+)?)")    
]

In [21]:
after_extract = {}

for ticker in tqdm(tkts):
    after_extract[ticker] = {}    
    for patt in range(len(patterns)):
        after_extract[ticker][patt+1] = ['-','-','-','-','-','-','-','-','-','-','-','-']
    
    if ticker in specific_tkts:
        for patt in range(len(specific_patterns)):
            after_extract[ticker][patt+22] = ['-','-','-','-','-','-','-','-','-','-','-','-']

100%|██████████| 505/505 [00:00<00:00, 14356.27it/s]


In [22]:
for ticker in tqdm(tkts):  
    if ticker in specific_tkts:
        for spec_patt in range(len(specific_patterns)):
            for line_income in range(len(res_list_income[ticker])):
                if (re.search(specific_patterns[spec_patt], res_list_income[ticker][line_income])) is not None:
                    temp_income = re.search(specific_patterns[spec_patt], res_list_income[ticker][line_income]).group().split()
                    after_extract[ticker][spec_patt+22] = temp_income
                    
            for line_balance in range(len(res_list_balance[ticker])):
                if (re.search(specific_patterns[spec_patt], res_list_balance[ticker][line_balance])) is not None:
                    temp_balance = re.search(specific_patterns[spec_patt], res_list_balance[ticker][line_balance]).group().split()
                    after_extract[ticker][spec_patt+22] = temp_balance  
                    
    for patt in range(len(patterns)):
        for line_income in range(len(res_list_income[ticker])):
            if (re.search(patterns[patt], res_list_income[ticker][line_income])) is not None:
                temp_income = re.search(patterns[patt], res_list_income[ticker][line_income]).group().split()
                after_extract[ticker][patt+1] = temp_income
            
        for line_balance in range(len(res_list_balance[ticker])):
            if (re.search(patterns[patt], res_list_balance[ticker][line_balance])) is not None:
                temp_balance = re.search(patterns[patt], res_list_balance[ticker][line_balance]).group().split()
                after_extract[ticker][patt+1] = temp_balance
            
        for line_cash in range(len(res_list_cash[ticker])):
            if (re.search(patterns[patt], res_list_cash[ticker][line_cash])) is not None:
                temp_cash = re.search(patterns[patt], res_list_cash[ticker][line_cash]).group().split()
                after_extract[ticker][patt+1] = temp_cash   

100%|██████████| 505/505 [00:01<00:00, 321.99it/s]


In [23]:
df_res = pd.DataFrame(columns=['Indicator', '2020','2019','2018','2017'])

indicators = ['revenue','cost of revenue','total assets','net income',
              'Accumulated Depreciation, Total','capital expenditures',
              'total revenue','total current assets','total current liabilities',
              'Cash and Short Term Investments','Notes Payable/Short Term Debt',
              'Payable/Accrued','Other Liabilities','Net Income Before Taxes',
              'Changes in Working Capital','retained earnings','Diluted Normalized EPS',
              'Total Equity','Total Common Shares Outstanding', 'Goodwill, Net', 'Intangibles, Net']

specific_indicators = ['revenue','cost of revenue','total assets','net income',
              'Accumulated Depreciation, Total','capital expenditures',
              'total revenue','total current assets','total current liabilities',
              'Cash and Short Term Investments','Notes Payable/Short Term Debt',
              'Payable/Accrued','Other Liabilities','Net Income Before Taxes',
              'Changes in Working Capital','retained earnings','Diluted Normalized EPS',
              'Total Equity','Total Common Shares Outstanding', 'Goodwill, Net', 'Intangibles, Net',
              'Interest Income, Bank', 'Total Interest Expense', 'Non-Interest Income, Bank', 
              'Non-Interest Expense, Bank', 'Total Liabilities', 'Cash & Due from Banks', 
              'Other Earning Assets, Total', 'Net Loans']

for t in tqdm(tkts):
    if t in specific_tkts:
        
        ind = [t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t]
        d = {'Indicator': specific_indicators,
         '2020':[after_extract[t][1][1],after_extract[t][2][4],after_extract[t][3][2],after_extract[t][4][2],after_extract[t][5][3],after_extract[t][6][2],after_extract[t][7][2],after_extract[t][8][3],after_extract[t][9][3],after_extract[t][10][5],after_extract[t][11][4],after_extract[t][12][1],after_extract[t][13][3],after_extract[t][14][4],after_extract[t][15][4],after_extract[t][16][4],after_extract[t][17][3],after_extract[t][18][2],after_extract[t][19][4],after_extract[t][20][2],after_extract[t][21][2],after_extract[t][22][3],after_extract[t][23][3],after_extract[t][24][3],after_extract[t][25][3],after_extract[t][26][2],after_extract[t][27][5],after_extract[t][28][4],after_extract[t][29][2]],
         '2019':[after_extract[t][1][2],after_extract[t][2][5],after_extract[t][3][3],after_extract[t][4][3],after_extract[t][5][4],after_extract[t][6][3],after_extract[t][7][3],after_extract[t][8][4],after_extract[t][9][4],after_extract[t][10][6],after_extract[t][11][5],after_extract[t][12][2],after_extract[t][13][4],after_extract[t][14][5],after_extract[t][15][5],after_extract[t][16][5],after_extract[t][17][4],after_extract[t][18][3],after_extract[t][19][5],after_extract[t][20][3],after_extract[t][21][3],after_extract[t][22][4],after_extract[t][23][4],after_extract[t][24][4],after_extract[t][25][4],after_extract[t][26][3],after_extract[t][27][6],after_extract[t][28][5],after_extract[t][29][3]], 
         '2018':[after_extract[t][1][3],after_extract[t][2][6],after_extract[t][3][4],after_extract[t][4][4],after_extract[t][5][5],after_extract[t][6][4],after_extract[t][7][4],after_extract[t][8][5],after_extract[t][9][5],after_extract[t][10][7],after_extract[t][11][6],after_extract[t][12][3],after_extract[t][13][5],after_extract[t][14][6],after_extract[t][15][6],after_extract[t][16][6],after_extract[t][17][5],after_extract[t][18][4],after_extract[t][19][6],after_extract[t][20][4],after_extract[t][21][4],after_extract[t][22][5],after_extract[t][23][5],after_extract[t][24][5],after_extract[t][25][5],after_extract[t][26][4],after_extract[t][27][7],after_extract[t][28][6],after_extract[t][29][4]], 
         '2017':[after_extract[t][1][4],after_extract[t][2][7],after_extract[t][3][5],after_extract[t][4][5],after_extract[t][5][6],after_extract[t][6][5],after_extract[t][7][5],after_extract[t][8][6],after_extract[t][9][6],after_extract[t][10][8],after_extract[t][11][7],after_extract[t][12][4],after_extract[t][13][6],after_extract[t][14][7],after_extract[t][15][7],after_extract[t][16][7],after_extract[t][17][6],after_extract[t][18][5],after_extract[t][19][7],after_extract[t][20][5],after_extract[t][21][5],after_extract[t][22][6],after_extract[t][23][6],after_extract[t][24][6],after_extract[t][25][6],after_extract[t][26][5],after_extract[t][27][8],after_extract[t][28][7],after_extract[t][29][5]]
        }
        
    else:
        ind = [t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t,t]
        d = {'Indicator': indicators,
             '2020':[after_extract[t][1][1],after_extract[t][2][4],after_extract[t][3][2],after_extract[t][4][2],after_extract[t][5][3],after_extract[t][6][2],after_extract[t][7][2],after_extract[t][8][3],after_extract[t][9][3],after_extract[t][10][5],after_extract[t][11][4],after_extract[t][12][1],after_extract[t][13][3],after_extract[t][14][4],after_extract[t][15][4],after_extract[t][16][4],after_extract[t][17][3],after_extract[t][18][2],after_extract[t][19][4],after_extract[t][20][2],after_extract[t][21][2]],
             '2019':[after_extract[t][1][2],after_extract[t][2][5],after_extract[t][3][3],after_extract[t][4][3],after_extract[t][5][4],after_extract[t][6][3],after_extract[t][7][3],after_extract[t][8][4],after_extract[t][9][4],after_extract[t][10][6],after_extract[t][11][5],after_extract[t][12][2],after_extract[t][13][4],after_extract[t][14][5],after_extract[t][15][5],after_extract[t][16][5],after_extract[t][17][4],after_extract[t][18][3],after_extract[t][19][5],after_extract[t][20][3],after_extract[t][21][3]], 
             '2018':[after_extract[t][1][3],after_extract[t][2][6],after_extract[t][3][4],after_extract[t][4][4],after_extract[t][5][5],after_extract[t][6][4],after_extract[t][7][4],after_extract[t][8][5],after_extract[t][9][5],after_extract[t][10][7],after_extract[t][11][6],after_extract[t][12][3],after_extract[t][13][5],after_extract[t][14][6],after_extract[t][15][6],after_extract[t][16][6],after_extract[t][17][5],after_extract[t][18][4],after_extract[t][19][6],after_extract[t][20][4],after_extract[t][21][4]], 
             '2017':[after_extract[t][1][4],after_extract[t][2][7],after_extract[t][3][5],after_extract[t][4][5],after_extract[t][5][6],after_extract[t][6][5],after_extract[t][7][5],after_extract[t][8][6],after_extract[t][9][6],after_extract[t][10][8],after_extract[t][11][7],after_extract[t][12][4],after_extract[t][13][6],after_extract[t][14][7],after_extract[t][15][7],after_extract[t][16][7],after_extract[t][17][6],after_extract[t][18][5],after_extract[t][19][7],after_extract[t][20][5],after_extract[t][21][5]]
            }
    df_temp = pd.DataFrame(d, index=ind) 
    df_res = df_res.append(df_temp)

100%|██████████| 505/505 [00:01<00:00, 421.39it/s]


In [24]:
df_res['FTV' : 'FTV']

,Indicator,2020,2019,2018,2017
FTV,revenue,1299.5,1319.7,1259.2,4634.4
FTV,cost of revenue,555.3,564.2,547.3,2025.9
FTV,total assets,14976.1,14495.5,14278.9,16051.5
FTV,net income,150.8,180.9,110.2,1613.3
FTV,"Accumulated Depreciation, Total",-701.1,-696.2,-680,-674.5
FTV,capital expenditures,-28,-17.1,-8.4,-75.7
FTV,total revenue,1299.5,1319.7,1259.2,4634.4
FTV,total current assets,2425.3,3086.4,2800.9,4446.9
FTV,total current liabilities,2868.9,2569.7,2474.8,2860.8
FTV,Cash and Short Term Investments,838.4,1565.8,1299.6,2944


In [25]:
df_res.to_excel('tables_usa_qf_Quarterly.xlsx')

In [ ]:
# # для добавления вручную, если некоторые тикеры отсутствуют после загрузки

# bad_ticker = 'FTV'

# driver = webdriver.Firefox()
# extension_dir = (os.path.join(os.path.expandvars("%userprofile%")) + '\\AppData\\Roaming\\Mozilla\\Firefox\\Profiles\\vg3v8nm4.default-release\\extensions\\')
# extensions = [
#     'uBlock0@raymondhill.net.xpi',
#     ]
# for extension in extensions:
#     driver.install_addon(extension_dir + extension, temporary=None)
        
# print("driver ready")  

# driver.get(getIncomeStatement(links[bad_ticker]))
# try:
#     driver.find_element_by_xpath('/html/body/div[5]/section/div[8]/div[1]/a[1]').click()
# except: 
#     driver.find_element_by_xpath('/html/body/div[5]/section/div[9]/div[1]/a[1]').click()
        
# time.sleep(3)
# res_list_income[bad_ticker] = driver.find_elements_by_xpath('/html/body/div[5]/section/div[9]/table/tbody[2]/tr')

# n = 0
# for el_inc in res_list_income[bad_ticker]:
#     res_list_income[bad_ticker][n] = el_inc.text
#     n+=1
    
# driver.close()